In [2]:
import json
import requests
import os
import pprint
import pandas
import time
from yaspin import yaspin
pp = pprint.PrettyPrinter()
%matplotlib widget

ModuleNotFoundError: No module named 'pandas'

In [2]:

TFE_ADDR =  os.environ.get('TFE_ADDR', 'https://app.terraform.io')
TFE_API_TOKEN =  os.environ.get('TFE_API_TOKEN', '')
TFE_SITE_ADMIN =  os.environ.get('TFE_SITE_ADMIN', 'false')
headers = {
    'Content-Type': 'application/vnd.api+json',
    'Authorization': f"Bearer {TFE_API_TOKEN}"  
    }

In [14]:
@yaspin(text="Loading...")
def getOrgObj():
    result = []
    resp = requests.get(TFE_ADDR+'/api/v2/organizations', headers=headers )
    if resp.status_code != 200:
        # This means something went wrong.
        pp.pprint(f"something went wrong: {resp}")
        return ""
    todo_item = resp.json()['data']
    for item in todo_item:
         info = { 
         'name': item['id'],
         'total_runs': getRunsTotalCount(item['id']),
         'workspace_count': getWorkspaceCount(item['id']),
         'workspaces': getWorkspaceInfo(item['id'])
         }
         result.append(info)
    return result    

def getRunsTotalCount(organisation):
    total_runs = 0
    todo_item = getWorkspaceInfo(organisation)
    for item in todo_item:
        total_runs +=  getRunsCount(item)
    pp.pprint(f"Total runs for {organisation}: {total_runs}")     
    return total_runs     

def getWorkspaceInfo(organisation):
    page = 1
    retorno = []
    resp = requests.get(TFE_ADDR+f'/api/v2/organizations/{organisation}/workspaces?page[size]=100&page[number]={page}', headers=headers )
    if resp.status_code != 200:
        # This means something went wrong.
        pp.pprint(f"something went wrong: {resp}")
        return ""
    metdata =   resp.json()['meta']['pagination']
    current_page = metdata["current-page"]
    total_pages = metdata["total-pages"]
    while page <= total_pages:
        todo_item = resp.json()['data']
        metdata =   resp.json()['meta']['pagination']
        current_page = metdata["current-page"]
        for item in todo_item:
            retorno.append(item['id'])
        page += 1
        resp = requests.get(TFE_ADDR+f'/api/v2/organizations/{organisation}/workspaces?page[size]=100&page[number]={page}', headers=headers )
    return retorno


def getWorkspaceCount(organisation):
    page = 1
    retorno = []
    resp = requests.get(TFE_ADDR+f'/api/v2/organizations/{organisation}/workspaces?page[size]=100&page[number]={page}', headers=headers )
    if resp.status_code != 200:
        # This means something went wrong.
        pp.pprint(f"something went wrong: {resp}")
        return ""
    metdata =   resp.json()['meta']['pagination']
    current_page = metdata["current-page"]
    total_pages = metdata["total-pages"]
    while page <= total_pages:
        todo_item = resp.json()['data']
        metdata =   resp.json()['meta']['pagination']
        current_page = metdata["current-page"]
        for item in todo_item:
            retorno.append(item['id'])
        page += 1
        resp = requests.get(TFE_ADDR+f'/api/v2/organizations/{organisation}/workspaces?page[size]=100&page[number]={page}', headers=headers )
    return len(retorno)

def getRunsCount(workspace_id):
    page = 1
    retorno = []
    resp = requests.get(TFE_ADDR+f'/api/v2/workspaces/{workspace_id}/runs?page[size]=100&page[number]={page}', headers=headers )
    if resp.status_code != 200:
        # This means something went wrong.
        pp.pprint(f"something went wrong: {resp}")
        return ""
    todo_item = resp.json()['data']
    metdata =   resp.json()['meta']['pagination']
    current_page = metdata["current-page"]
    total_pages = metdata["total-pages"]
    while page <= total_pages:
        todo_item = resp.json()['data']
        metdata =   resp.json()['meta']['pagination']
        current_page = metdata["current-page"]
        for item in todo_item:
            retorno.append(item['id'])
        page += 1
        resp = requests.get(TFE_ADDR+f'/api/v2/workspaces/{workspace_id}/runs?page[size]=100&page[number]={page}', headers=headers )
    return len(retorno)
    

In [16]:
data = pandas.DataFrame(getOrgObj())
data

⠇ Loading...'Total runs for andygriffindemo: 24'
⠋ Loading...'Total runs for BT-Demo: 47'
⠏ Loading...'Total runs for DELOITTE_TFE_TRIAL: 6'
⠴ Loading...'Total runs for devworks: 132'
⠼ Loading...'Total runs for dlectronique: 71'
⠇ Loading...'Total runs for emea-se-playground-2019: 3451'
⠇ Loading...'Total runs for example-organization: 247'
⠴ Loading...'Total runs for G_RESEARCH_DEMO: 20'
⠹ Loading...'Total runs for GuyBarros: 1'
⠹ Loading...'Total runs for Hashicorp-neh-Demo: 261'
⠸ Loading...'Total runs for hashicorp-se-demos: 36'
⠏ Loading...'Total runs for hc-emea-sentinel-demo: 102'
⠸ Loading...'Total runs for hc-solutions-engineering: 1838'
⠦ Loading...'Total runs for jpapazian-org: 120'
⠇ Loading...'Total runs for Microsoft-UK-TechHurdle-Trial: 23'
⠋ Loading...'Total runs for rgustafsson: 171'
⠧ Loading...'Total runs for RogerBerlind: 1413'
⠧ Loading...'Total runs for rusty-pro: 371'
⠹ Loading...'Total runs for SKY_OTT_VAULT_POV: 20'
⠴ Loading...'Total runs for SNOW-se-demos: 9

,name,total_runs,workspace_count,workspaces
0,andygriffindemo,24,2,"[ws-jiBewknXB6XC4eL7, ws-yuoQbRT5BtS2TpbZ]"
1,BT-Demo,47,1,[ws-6qqQbs1C9Dy5UEKj]
2,DELOITTE_TFE_TRIAL,6,1,[ws-dSNRjCPLEAXa5wFT]
3,devworks,132,7,"[ws-U1PCiFEmBLXcQyFn, ws-3N1ss6DBgP9vRSB6, ws-..."
4,dlectronique,71,5,"[ws-TpHm1VbN7Rk12n1t, ws-YdNNT7niaePkKW5x, ws-..."
5,emea-se-playground-2019,3451,89,"[ws-NNPCZRmKM6AvB1uE, ws-6LvLRx7uFoDD7Znf, ws-..."
6,example-organization,247,12,"[ws-z16LcBbm9wSBpF54, ws-r8GQgRHNXaokQvkf, ws-..."
7,G_RESEARCH_DEMO,20,3,"[ws-7ZyZtKjf6xv6ecPA, ws-WZsfMBSzsvy7SpvN, ws-..."
8,GuyBarros,1,6,"[ws-SeuUKkNk68LuPJ8s, ws-QbbCBd1nsc4bsDtx, ws-..."
9,Hashicorp-neh-Demo,261,32,"[ws-SxZJAeLU3VExzv3b, ws-ypBjTJcq6r1T5yAX, ws-..."


In [17]:
df = data[['name','total_runs']]

In [19]:
ax = df.head().plot.bar(x='name', y='total_runs', rot=0)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
a